In [11]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

cats = ['AI','LC','LST','NDBSI','NDVI','WET']
pcs = ['pc01','pc02','pc03','pc04','pc05','pc06']
results = pd.DataFrame()
percents = pd.DataFrame()

df = pd.read_excel('D:\ee-composite\src\sqlserver\Data_E_1.xlsx')
# print(df,df.shape)

df1 = df.query('catid=="AI"').sort_values('sn')
for yyyy in range(1985,2021):
    x = df1.loc[:,['sn','cityen','province','city']]
    x['year'] = yyyy
    results = results.append(x)
results = results.reset_index()   
results.drop('index', axis=1, inplace=True) 
# print(results,results.shape)

for i in range(1,7,1):
    cc = cats[i-1]
    df1 = df.query('catid=="'+cc+'"').sort_values('sn')
    tmp = pd.DataFrame()
    for yyyy in range(1985,2021):
        tmp = tmp.append(df1.loc[:,[yyyy]].rename(columns={yyyy:cc}))
    tmp = tmp.reset_index()[cc]
    tmp = (tmp-tmp.mean())/tmp.std()
    tmp = (tmp-tmp.min())/(tmp.max()-tmp.min())
    results = pd.concat([results,tmp],axis=1)
# print(results,results.shape)

for i in x['sn']:
    pcadata = results.query('sn=='+str(i)).sort_values('year').iloc[:,5:]
    # print(pcadata,pcadata.shape)

    pca = PCA(n_components=6,whiten=False)
    tmp = pca.fit_transform(pcadata)
    pcp =  pd.DataFrame(pca.explained_variance_ratio_).T
    pcp.columns = pcs
    # print(pcp,pcp.shape)
    pcp['sn'] = pd.Series([i])
    pcp['city'] = pd.Series([x.iat[i-1,1]])
    pcp['citycn'] = pd.Series(x.iat[i-1,3])
    pcp['province'] = pd.Series(x.iat[i-1,2])
    percents = pd.concat([percents,pcp])

print(percents,percents.shape)
print(percents.mean(),percents.std())
print(percents.groupby(['sn','city','citycn']).mean())

percents.groupby(['sn','city','citycn']).mean().to_excel('Data_E_1_explain.xlsx', sheet_name='explain')

       pc01      pc02      pc03      pc04      pc05      pc06  sn  \
0  0.945494  0.043224  0.008642  0.002382  0.000219  0.000039   1   
0  0.839397  0.132441  0.024865  0.003109  0.000171  0.000017   2   
0  0.654188  0.294242  0.047104  0.004427  0.000034  0.000005   3   
0  0.948798  0.038459  0.008648  0.003828  0.000231  0.000037   4   
0  0.903612  0.078802  0.014873  0.002155  0.000525  0.000034   5   
0  0.679195  0.248136  0.066782  0.005606  0.000255  0.000025   6   
0  0.875696  0.105910  0.015370  0.002776  0.000189  0.000059   7   
0  0.710166  0.242650  0.041791  0.004817  0.000489  0.000087   8   
0  0.878955  0.100025  0.018326  0.002624  0.000061  0.000009   9   
0  0.937095  0.035601  0.020051  0.007156  0.000084  0.000013  10   
0  0.915112  0.072535  0.008099  0.003028  0.001202  0.000025  11   
0  0.918633  0.054356  0.019845  0.005673  0.001174  0.000319  12   
0  0.792216  0.184085  0.020451  0.003203  0.000039  0.000005  13   
0  0.929929  0.055290  0.008323  0